[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/myui/rtrec/blob/main/notebooks/quickstart_colab.ipynb)

# Install rtrec library using PIP

In [ ]:
!pip install rtrec

# Prepare data for Training and Evaluation

In [ ]:
from rtrec.experiments.datasets import load_dataset

# load Movielens 1M dataset
df = load_dataset(name='movielens_1m')

In [ ]:
# Confirm dataframe data
df[['user', 'item', 'tstamp', 'rating']]

In [ ]:
# Split data set by temporal user split
# Use 70% for training and 30% for test
from rtrec.experiments.split import temporal_user_split
train_df, test_df = temporal_user_split(df, test_frac=0.3)

# Use SLIM with Feature Selection as the Model

In [ ]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

#model = SLIM() 
model = SLIM(nn_feature_selection=50) 
#model = SLIM(min_value=0, max_value=15, decay_in_days=180, nn_feature_selection=50)

recommender = Recommender(model)

In [ ]:
# Bulk fit
recommender.bulk_fit(train_df)

In [ ]:
# Evaluation
metrics = recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)
print(metrics)

# Incremental Fit

In [ ]:
from rtrec.experiments.split import temporal_split
# Split test data into two
test_df1, test_df2 = temporal_split(test_df, test_frac=0.5)

# Incremental fit
recommender.fit(test_df1, update_interaction=True, parallel=True)

In [ ]:
# Evaluation to confirm incremental fit works
metrics = recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)
print(metrics)

# User to Item Recommendation

In [ ]:
recommended = recommender.recommend(user=10, top_k=10, filter_interacted=True)
assert len(recommended) == 10

# Item to Item Recommendation

In [ ]:
# Item to Item recommendation
similar_items = recommender.similar_items(query_items=[3,10], top_k=5)
assert len(similar_items) == 2

In [ ]:
test_df2.head(5)

# Play with Raw-level APIs

In [ ]:
import time
current_unixtime = time.time()
interactions = [('user_1', 'item_1', current_unixtime, 5.0),
                ('user_2', 'item_2', current_unixtime, -2.0),
                ('user_2', 'item_1', current_unixtime, 3.0),
                ('user_2', 'item_4', current_unixtime, 3.0),
                ('user_1', 'item_3', current_unixtime, 4.0)]

interactions

In [ ]:
# Fit SLIM model
from rtrec.models import SLIM
model = SLIM()
model.fit(interactions)

In [ ]:
# can fit from streams using yield as follows:
def yield_interactions():
    for interaction in interactions:
        yield interaction
model.fit(yield_interactions())

In [ ]:
# Recommend top-5 items for a user
recommendations = model.recommend('user_1', top_k=5)
assert recommendations == ["item_4", "item_2"]